In [ ]:
import ipywidgets as widgets
from ipywidgets import GridspecLayout, Output
from IPython.display import display, clear_output, HTML
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
import seaborn as sns
import pandas as pd
from pandas.api.types import CategoricalDtype
import textwrap
import numpy as np

# Configure the plotting
plt.style.use('seaborn-v0_8-darkgrid')

# Load the dataset
# Detect if running in Google Colab
try:
    import google.colab
    url = "https://raw.githubusercontent.com/ChokZB/hr_attrition_dashboard/HEAD/data/hr_analytics.csv"
    df = pd.read_csv(url)
    # print("Running in Colab → Loaded dataset from GitHub.")
except ImportError:
    df = pd.read_csv("data/hr_analytics.csv")
    # print("Running locally → Loaded dataset from /data folder.")

# # Display the first few rows of the dataset to understand its structure
# df.T

# # Check the unique value in all columns
# for column in df.columns:
#     print(column)
#     print(df[column].unique())
#     print()

# Drop the unnecessary columns
df.drop(['EmpID', 'EducationField', 'EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours'], axis=1, inplace=True)

# Rename columns for better readability using a single rename function call
df.rename(columns={
    'AgeGroup': 'Age Group',
    'BusinessTravel': 'Business Travel',
    'DailyRate': 'Daily Rate',
    'DistanceFromHome': 'Distance from Home',
    'Education': 'Education Level',
    'EnvironmentSatisfaction': 'Environment Satisfaction',
    'HourlyRate': 'Hourly Rate',
    'JobInvolvement': 'Job Involvement',
    'JobLevel': 'Job Level',
    'JobRole': 'Job Role',
    'JobSatisfaction': 'Job Satisfaction',
    'MaritalStatus': 'Marital Status',
    'MonthlyIncome': 'Monthly Income',
    'SalarySlab': 'Salary Slab',
    'MonthlyRate': 'Monthly Rate',
    'NumCompaniesWorked': 'Number of Companies Worked',
    'OverTime': 'Over Time',
    'PercentSalaryHike': 'Percent Salary Hike',
    'PerformanceRating': 'Performance Rating',
    'RelationshipSatisfaction': 'Relationship Satisfaction',
    'StockOptionLevel': 'Stock Option Level',
    'TotalWorkingYears': 'Total Working Years',
    'TrainingTimesLastYear': 'Training Times Last Year',
    'WorkLifeBalance': 'Work Life Balance',
    'YearsAtCompany': 'Years at Company',
    'YearsInCurrentRole': 'Years in Current Role',
    'YearsSinceLastPromotion': 'Years Since Last Promotion',
    'YearsWithCurrManager': 'Years with Current Manager'
}, inplace=True)

# # Display the first few rows to verify the column names
# df.T

# Define the function to categorize 'Distance From Home'
def categorize_distance(distance):
    if distance < 5:
        return '<5 km'
    elif 5 <= distance <= 15:
        return '5-15 km'
    else:
        return '>15 km'

# Define the function to categorize 'Performance Rating'
def categorize_performance(rating):
    if rating == 3:
        return 'Low Performance'
    elif rating == 4:
        return 'High Performance'

# Apply the function to the 'Distance from Home' column
df['Distance from Home'] = df['Distance from Home'].apply(categorize_distance)

# Apply the function to the 'Performance Rating' column
df['Performance Rating'] = df['Performance Rating'].apply(categorize_performance)

# Rename values in 'Business Travel'
df['Business Travel'] = df['Business Travel'].replace({
    'TravelRarely': 'Travel Rarely',
    'Travel_Rarely': 'Travel Rarely',
    'Travel_Frequently': 'Travel Frequently'
})

# Rename values in 'Department'
df['Department'] = df['Department'].replace({
    'Research & Development': 'R&D'
})

# Rename values in 'Education'
df['Education Level'] = df['Education Level'].replace({
    1: "High School",
    2: "Associate's Degree",
    3: "Bachelor's Degree",
    4: "Master's Degree",
    5: "Doctoral Degree"
})

# Rename values in 'Salary Slab'
df['Salary Slab'] = df['Salary Slab'].replace({
    'Upto 5k': 'Up to 5k'
})

education_order = ['High School',
                   "Associate's Degree",
                   "Bachelor's Degree",
                   "Master's Degree",
                   'Doctoral Degree']

education_dtype = CategoricalDtype(categories=education_order, ordered=True)

# Apply to the whole dataframe once
df['Education Level'] = df['Education Level'].astype(education_dtype)


# # Check the unique value in all columns
# for column in df.columns:
#     print(column)
#     print(df[column].unique())
#     print()

# # Check the dimensions of the dataset
# df.shape

# # Check the missing values in the dataset
# df.isnull().sum()

# Drop the missing values from the dataset
df.dropna(inplace=True)

# # Check the dimensions of the dataset after dropping the missing values
# df.shape

# Check the data types of the columns in the dataset
df.dtypes

# Dictionary to map columns to their new data types
dtype_changes = {
    'Age Group': 'category',
    'Attrition': 'category',
    'Business Travel': 'category',
    'Distance from Home': 'category',
    'Department': 'category',
    # 'Education Level': 'category',
    'Environment Satisfaction': 'category',
    'Gender': 'category',
    'Job Involvement': 'category',
    'Job Level': 'category',
    'Job Role': 'category',
    'Job Satisfaction': 'category',
    'Marital Status': 'category',
    'Salary Slab': 'category',
    'Over Time': 'category',
    'Performance Rating': 'category',
    'Relationship Satisfaction': 'category',
    'Stock Option Level': 'category',
    'Work Life Balance': 'category',    
    'Years with Current Manager': 'int64'
}
# Change the data types
for column, dtype in dtype_changes.items():
    df[column] = df[column].astype(dtype)

# # Check the data types of the columns in the dataset after the changes
# df.dtypes

# Function to convert the colormap to CSS RGBA
def colormap_to_css_rgba(cmap, index):
    norm = Normalize(vmin=0, vmax=255)
    mappable = ScalarMappable(norm=norm, cmap=cmap)
    rgba = mappable.to_rgba(index, bytes=True)
    return f'rgba({rgba[0]}, {rgba[1]}, {rgba[2]}, {rgba[3] / 255})'

# Get the colors from the 'Purples' colormap
purple_color_1 = colormap_to_css_rgba('Purples', 128)
purple_color_2 = colormap_to_css_rgba('Purples', 200)
hover_purple = colormap_to_css_rgba('Purples', 150)

# Function to create CSS styles
def create_css_styles(purple_color_1, purple_color_2, hover_purple):
    css = f"""
    <style>
    .widget-dropdown .widget-label {{
        background-color: {purple_color_2} !important;
        color: white !important;
        text-align: center !important;
        border-radius: 5px !important;
        width: auto !important; 
        padding: 0 10px !important;
    }}
    .widget-dropdown select {{
        background-color: {purple_color_2} !important;
        color: white !important;
        border-radius: 5px !important;
        -webkit-appearance: none;
        -moz-appearance: none;
        appearance: none;
        padding-right: 1.5em;
        background-image: url('data:image/svg+xml;utf8,<svg xmlns="http://www.w3.org/2000/svg" width="10" height="5" viewBox="0 0 10 5"><path fill="white" d="M0 0l5 5 5-5z"/></svg>');
        background-repeat: no-repeat;
        background-position: right 10px center;
        background-size: 10px 5px;
    }}
    .widget-dropdown select::-ms-expand {{
        display: none;
    }}
    .widget-dropdown select option {{
        background-color: white !important;
        color: black !important;
    }}
    .widget-dropdown select option:hover {{
        background-color: {hover_purple} !important;
        color: white !important;
    }}
    .widget-button {{
        background-color: {purple_color_2} !important;
        color: white !important;
        border-radius: 5px !important;
        width: auto !important; 
        padding: 0 10px !important;
    }}
    .widget-slider .widget-label {{
        background-color: {purple_color_2} !important;
        color: white !important;
        text-align: center !important;
        border-radius: 5px !important;
        width: auto !important; 
        padding: 0 10px !important;
    }}
    .widget-slider .noUi-base {{
        background: {purple_color_2} !important;
    }}
    .widget-slider .noUi-connect {{
        background: {purple_color_2} !important;
    }}
    .widget-slider .noUi-handle {{
        background: white !important;
        border: 2px solid {purple_color_2} !important;
    }}
    .widget-slider .noUi-horizontal .noUi-handle {{
        width: 18px;
        height: 18px;
        border-radius: 50%;
        top: -6px;
    }}
    .widget-checkbox .widget-label {{
        background-color: {purple_color_2} !important;
        color: white !important;
        text-align: center !important;
        border-radius: 5px !important;
        padding: 5px;
    }}
    .widget-checkbox input[type="checkbox"] {{
        accent-color: {purple_color_2} !important;
        margin-right: 5px;
    }}
    .widget-checkbox input[type="checkbox"]:hover {{
        accent-color: {hover_purple} !important;
    }}
    </style>
    """
    return css

# Display the CSS
display(HTML(create_css_styles(purple_color_1, purple_color_2, hover_purple)))

# Card function
def create_cards(df):
    total_employees = len(df)
    total_active_employees = len(df[df['Attrition'] == 'No'])
    total_attrition = len(df[df['Attrition'] == 'Yes'])
    total_attrition_rate = (total_attrition / total_employees) * 100 if total_employees > 0 else 0

    cards_html = f"""
    <br>
    <div style="display: flex; justify-content: space-around; margin: 10px;">
        <div style="background-color: {purple_color_1}; color: white; padding: 10px; border-radius: 5px; width: 230px; text-align: center;">
            <h3 style="font-size: 1.5em;">Total Employees:</h3>
            <p style="font-size: 1.3em; margin: 0; text-align: center;">{total_employees}</p>
        </div>
        <div style="background-color: {purple_color_1}; color: white; padding: 10px; border-radius: 5px; width: 230px; text-align: center;">
            <h3 style="font-size: 1.5em;">Total Active Employees:</h3>
            <p style="font-size: 1.3em; margin: 0; text-align: center;">{total_active_employees}</p>
        </div>
        <div style="background-color: {purple_color_1}; color: white; padding: 10px; border-radius: 5px; width: 230px; text-align: center;">
            <h3 style="font-size: 1.5em;">Total Attrition:</h3>
            <p style="font-size: 1.3em; margin: 0; text-align: center;">{total_attrition}</p>
        </div>
        <div style="background-color: {purple_color_1}; color: white; padding: 10px; border-radius: 5px; width: 230px; text-align: center;">
            <h3 style="font-size: 1.5em;">Total Attrition Rate:</h3>
            <p style="font-size: 1.3em; margin: 0; text-align: center;">{total_attrition_rate:.2f}%</p>
        </div>
    </div>
    <br>
    """
    display(HTML(cards_html))

# Filter for department
department_filter = widgets.Dropdown(
    options=['All'] + list(df['Department'].unique()),
    value='All',
    description='Department:',
    layout=widgets.Layout(width='auto'),
    style={'description_width': 'initial'}
)

# Checkboxes for Gender
gender_checkboxes = []
for gender in df['Gender'].unique():
    gender_checkboxes.append(widgets.Checkbox(value=True, description=gender, layout=widgets.Layout(width='auto', margin='0 10px 0 0'), indent=False))

# Create VBox for Gender checkboxes
gender_box = widgets.HBox(gender_checkboxes, layout=widgets.Layout(width='auto', align_items='center'))

# Function to handle checkbox changes
def on_gender_checkbox_change(change):
    checked_boxes = [chk for chk in gender_checkboxes if chk.value]

    if len(checked_boxes) == 0:
        # Automatically re-check the other checkbox
        if change.owner.description == 'Male':
            next_checkbox = [chk for chk in gender_checkboxes if chk.description == 'Female'][0]
        else:
            next_checkbox = [chk for chk in gender_checkboxes if chk.description == 'Male'][0]
        next_checkbox.value = True

# Slider for Age Range
age_slider = widgets.IntRangeSlider(
    value=[df['Age'].min(), df['Age'].max()],
    min=df['Age'].min(),
    max=df['Age'].max(),
    step=1,
    description='Age Range:',
    continuous_update=False,
)

# Button for Clear Filters
clear_filters = widgets.Button(
    description='Clear Filters',
    button_style='',
    layout=widgets.Layout(width='auto')
)

# Loading spinner
loading_spinner = widgets.HTML(
    value='<i class="fa fa-spinner fa-spin" style="font-size:24px; color: purple;"></i>',
    layout=widgets.Layout(visibility='hidden')
)

# Function to disable all widgets
def disable_widgets():
    department_filter.disabled = True
    age_slider.disabled = True
    clear_filters.disabled = True
    for checkbox in gender_checkboxes:
        checkbox.disabled = True

# Function to enable all widgets
def enable_widgets():
    department_filter.disabled = False
    age_slider.disabled = False
    clear_filters.disabled = False
    for checkbox in gender_checkboxes:
        checkbox.disabled = False

# Function to clear filters
def clear_filters_func(b):
    loading_spinner.layout.visibility = 'visible'
    disable_widgets()
    department_filter.value = 'All'
    age_slider.value = [df['Age'].min(), df['Age'].max()]
    for checkbox in gender_checkboxes:
        checkbox.value = True
    enable_widgets()
    update_dashboard()
    loading_spinner.layout.visibility = 'hidden'

clear_filters.on_click(clear_filters_func)

# Output widget for displaying the dashboard
output = widgets.Output()

def update_dashboard(*args):
    # Disable the widgets and show the spinner
    disable_widgets()
    loading_spinner.layout.visibility = 'visible'

    with output:
        clear_output(wait=True)
        filtered_df = df.copy()

        if department_filter.value != 'All':
            filtered_df = filtered_df[filtered_df['Department'] == department_filter.value]

        selected_genders = [chk.description for chk in gender_checkboxes if chk.value]
        if selected_genders:
            filtered_df = filtered_df[filtered_df['Gender'].isin(selected_genders)]

        age_range = age_slider.value
        filtered_df = filtered_df[(filtered_df['Age'] >= age_range[0]) & (filtered_df['Age'] <= age_range[1])]
        
        # Display title
        display(HTML(f"""
        <div style="text-align: center; margin-top: 20px;">
            <h1 style="font-size: 2em; background-color: {purple_color_2}; color: white; padding: 10px; border-radius: 10px;">HR ATTRITION DASHBOARD</h1>
            <br>
        </div>
        """))

        # Display filters in a horizontal layout
        filter_widgets = widgets.HBox(
            [department_filter, gender_box, age_slider, clear_filters, loading_spinner],
            layout=widgets.Layout(
                display='flex',
                flex_flow='row wrap',
                justify_content='space-around'
            )
        )
        display(filter_widgets)

        # Display cards
        create_cards(filtered_df)

        # 1st row
        plt.figure(figsize=(20, 5))

        ### 1st row 1st chart: Total Attrition by Department
        plt.subplot(1, 4, 1)

        # Filter data for attrition cases
        attrition_data = filtered_df[filtered_df['Attrition'] == 'Yes']

        # Count the occurrences of each category
        counts = attrition_data['Department'].value_counts()

        # Define the colormap and avoid the lightest shades
        cmap = plt.get_cmap('Purples')
        colors = [cmap(i / (len(counts) + 1)) for i in range(1, len(counts) + 1)]

        # Calculate total count for use in the lambda function
        total_count = counts.sum()

        # Create labels with percentages
        labels = [f'{count / total_count:.1%}' for count in counts.values]

        # Create pie chart
        wedges, texts, autotexts = plt.pie(
            counts, 
            labels=labels,
            autopct=lambda pct: f'{int(round(pct * total_count / 100.0))}', 
            colors=colors, 
            startangle=90, 
            pctdistance=0.85, 
            wedgeprops=dict(width=0.3, edgecolor='w')
        )

        # Draw a circle at the center to make it look like a donut chart
        centre_circle = plt.Circle((0, 0), 0.70, fc='white')
        fig = plt.gcf()
        fig.gca().add_artist(centre_circle)

        # Equal aspect ratio ensures that pie is drawn as a circle
        plt.tight_layout()

        # Wrap the title text to a new line if it exceeds the max_title_length
        wrapped_title = "\n".join(textwrap.wrap('Total Attrition by Department', 20))

        # Add annotations for the title in the center
        plt.text(0, 0, f'{wrapped_title}', ha='center', va='center', size=14, weight='bold')
            
        # Set color and weight for the autotexts inside the chart
        for autotext in autotexts:
            autotext.set_color('white')
            autotext.set_weight('bold')
            autotext.set_fontsize(12)

        # Add a legend
        plt.legend(wedges, counts.index, loc="upper center", bbox_to_anchor=(0.5, -0.05), ncol=len(counts))

        ### 1st row 2nd chart: Total Attrition by Distance from Home
        plt.subplot(1, 4, 2)

        # Filter data for attrition cases
        attrition_data = filtered_df[filtered_df['Attrition'] == 'Yes']
        
        # Count the occurrences of each category
        counts = attrition_data['Distance from Home'].value_counts()
        
        # Define the colormap and avoid the lightest shades
        cmap = plt.get_cmap('Purples')
        colors = [cmap(i / (len(counts) + 1)) for i in range(1, len(counts) + 1)]
        
        # Calculate total count for use in the lambda function
        total_count = counts.sum()

        # Create labels with percentages
        labels = [f'{count / total_count:.1%}' for count in counts.values]

        # Create pie chart
        wedges, texts, autotexts = plt.pie(
            counts, 
            labels=labels,
            autopct=lambda pct: f'{int(round(pct * total_count / 100.0))}', 
            colors=colors, 
            startangle=90, 
            pctdistance=0.85, 
            wedgeprops=dict(width=0.3, edgecolor='w')
        )
        
        # Draw a circle at the center to make it look like a donut chart
        centre_circle = plt.Circle((0, 0), 0.70, fc='white')
        fig = plt.gcf()
        fig.gca().add_artist(centre_circle)
        
        # Equal aspect ratio ensures that pie is drawn as a circle
        plt.tight_layout()

        # Wrap the title text to a new line if it exceeds the max_title_length
        wrapped_title = "\n".join(textwrap.wrap('Total Attrition by Distance from Home', 20))
        
        # Add annotations for the title in the center
        plt.text(0, 0, f'{wrapped_title}', ha='center', va='center', size=14, weight='bold')
            
        # Set color and weight for the autotexts inside the chart
        for autotext in autotexts:
            autotext.set_color('white')
            autotext.set_weight('bold')
            autotext.set_fontsize(12)
        
        # Add a legend
        plt.legend(wedges, counts.index, loc="upper center", bbox_to_anchor=(0.5, -0.05), ncol=len(counts))


        ### 1st row 3rd chart: Total Attrition by Salary Slab
        plt.subplot(1, 4, 3)

        # Filter data for attrition cases
        attrition_data = filtered_df[filtered_df['Attrition'] == 'Yes']
        
        # Count the occurrences of each category
        counts = attrition_data['Salary Slab'].value_counts()
        
        # Define the colormap and avoid the lightest shades
        cmap = plt.get_cmap('Purples')
        colors = [cmap(i / (len(counts) + 1)) for i in range(1, len(counts) + 1)]
        
        # Calculate total count for use in the lambda function
        total_count = counts.sum()

        # Create labels with percentages
        labels = [f'{count / total_count:.1%}' for count in counts.values]

        # Create pie chart
        wedges, texts, autotexts = plt.pie(
            counts, 
            labels=labels,
            autopct=lambda pct: f'{int(round(pct * total_count / 100.0))}', 
            colors=colors, 
            startangle=90, 
            pctdistance=0.85, 
            wedgeprops=dict(width=0.3, edgecolor='w')
        )
        
        # Draw a circle at the center to make it look like a donut chart
        centre_circle = plt.Circle((0, 0), 0.70, fc='white')
        fig = plt.gcf()
        fig.gca().add_artist(centre_circle)
        
        # Equal aspect ratio ensures that pie is drawn as a circle
        plt.tight_layout()

        # Wrap the title text to a new line if it exceeds the max_title_length
        wrapped_title = "\n".join(textwrap.wrap('Total Attrition by Salary Slab', 20))
        
        # Add annotations for the title in the center
        plt.text(0, 0, f'{wrapped_title}', ha='center', va='center', size=14, weight='bold')
            
        # Set color and weight for the autotexts inside the chart
        for autotext in autotexts:
            autotext.set_color('white')
            autotext.set_weight('bold')
            autotext.set_fontsize(12)
        
        # Add a legend
        plt.legend(wedges, counts.index, loc="upper center", bbox_to_anchor=(0.5, -0.05), ncol=len(counts))


        ### 1st row 4th chart: 
        plt.subplot(1, 4, 4)

        # Filter data for attrition cases
        attrition_data = filtered_df[filtered_df['Attrition'] == 'Yes']
        
        # Count the occurrences of each category
        counts = attrition_data['Performance Rating'].value_counts()
        
        # Define the colormap and avoid the lightest shades
        cmap = plt.get_cmap('Purples')
        colors = [cmap(i / (len(counts) + 1)) for i in range(1, len(counts) + 1)]
        
        # Calculate total count for use in the lambda function
        total_count = counts.sum()

        # Create labels with percentages
        labels = [f'{count / total_count:.1%}' for count in counts.values]

        # Create pie chart
        wedges, texts, autotexts = plt.pie(
            counts, 
            labels=labels,
            autopct=lambda pct: f'{int(round(pct * total_count / 100.0))}', 
            colors=colors, 
            startangle=90, 
            pctdistance=0.85, 
            wedgeprops=dict(width=0.3, edgecolor='w')
        )
        
        # Draw a circle at the center to make it look like a donut chart
        centre_circle = plt.Circle((0, 0), 0.70, fc='white')
        fig = plt.gcf()
        fig.gca().add_artist(centre_circle)
        
        # Equal aspect ratio ensures that pie is drawn as a circle
        plt.tight_layout()

        # Wrap the title text to a new line if it exceeds the max_title_length
        wrapped_title = "\n".join(textwrap.wrap('Total Attrition by Performance Rating', 20))
        
        # Add annotations for the title in the center
        plt.text(0, 0, f'{wrapped_title}', ha='center', va='center', size=14, weight='bold')
            
        # Set color and weight for the autotexts inside the chart
        for autotext in autotexts:
            autotext.set_color('white')
            autotext.set_weight('bold')
            autotext.set_fontsize(12)
        
        # Add a legend
        plt.legend(wedges, counts.index, loc="upper center", bbox_to_anchor=(0.5, -0.05), ncol=len(counts))


        plt.tight_layout()
        plt.show()


        # 2nd row
        plt.figure(figsize=(20, 8))

        ### 2nd row 1st chart: Total Attrition by Job Role
        plt.subplot(2, 3, 1)

        # Remove the background color for the plot
        plt.gca().patch.set_alpha(0)
        
        # Filter data for attrition cases
        attrition_data = filtered_df[filtered_df['Attrition'] == 'Yes']
        counts = attrition_data['Job Role'].value_counts().sort_values(ascending=False)
        
        # Define the colormap and avoid the lightest shades
        cmap = plt.get_cmap('Purples')
        colors = [cmap(i / (len(counts) + 1)) for i in range(1, len(counts) + 1)]
        color_mapping = dict(zip(counts.index, colors))
        
        plot = sns.barplot(x=counts.values, y=counts.index, palette=color_mapping, order=counts.index, hue=counts.index, dodge=False, legend=False)
        
        for p in plot.patches:
            width = p.get_width()
            if width > 0:
                plt.annotate(f'{int(width)}', (p.get_x() + width, p.get_y() + p.get_height() / 2), ha='left', va='center', xytext=(5, 0), textcoords='offset points')
            
        # Add title and labels
        plt.title('Total Attrition by Job Role')
        plt.ylabel('')

        # Remove x-ticks for a cleaner look
        plt.xticks([])  


        ### 2nd row 2nd chart: Total Attrition by Years Since Last Promotion
        plt.subplot(2, 3, 2)

        # Remove the background color for the plot
        plt.gca().patch.set_alpha(0)

        # Filter data for attrition cases
        attrition_data = filtered_df[filtered_df['Attrition'] == 'Yes']
        counts = attrition_data['Years Since Last Promotion'].value_counts().sort_index()
        
        # cmap = plt.get_cmap('Purples')
        colors = [cmap(i / (len(counts) + 1)) for i in range(1, len(counts) + 1)]
        
        plot = sns.barplot(x=counts.index, y=counts.values, palette=colors, hue=counts.index, dodge=False, legend=False)
        
        for p in plot.patches:
            height = p.get_height()
            if height > 0:  # Add annotation only if the height is greater than 0
                plot.annotate(f'{int(height)}', (p.get_x() + p.get_width() / 2., height), ha='center', va='center', xytext=(0, 10), textcoords='offset points')
        
        
        # Add title and labels
        plt.title('Total Attrition by Years Since Last Promotion')
        plt.xlabel('')
        plt.ylabel('')

        # Remove y-ticks for a cleaner look
        plt.yticks([])          
        

        ### 2nd row 3rd chart: Age Group & Gender Chart
        plt.subplot(2, 3, 3)

        # Remove the background color for the plot
        plt.gca().patch.set_alpha(0)

        # Filter data for attrition cases
        attrition_data = filtered_df[filtered_df['Attrition'] == 'Yes']

        # Create the count plot
        plot = sns.countplot(data=attrition_data, x='Age Group', hue='Gender', palette="Purples")

        # Add title and labels
        plt.title('Total Attrition by Age Group & Gender')
        plt.xlabel('')
        plt.ylabel('')

        # Remove y-ticks for a cleaner look
        plt.yticks([])

        # Add annotations to the bars
        for p in plot.patches:
            height = p.get_height()
            if height > 0:  # Add annotation only if the height is greater than 0
                plot.annotate(f'{int(height)}', (p.get_x() + p.get_width() / 2., height), ha='center', va='center', xytext=(0, 10), textcoords='offset points')

        ### 3rd row 1st chart: Total Attrition by Education Level
        plt.subplot(2, 3, 4)

        # Remove the background color for the plot
        plt.gca().patch.set_alpha(0)
        
        # Filter data for attrition cases
        attrition_data = filtered_df[filtered_df['Attrition'] == 'Yes']

        education_order = ['High School', "Associate's Degree", "Bachelor's Degree", "Master's Degree", 'Doctoral Degree']
        attrition_data = filtered_df[filtered_df['Attrition'] == 'Yes']
        counts = attrition_data['Education Level'].value_counts().sort_index()
        
        # Define the colormap and avoid the lightest shades
        cmap = plt.get_cmap('Purples')
        colors = [cmap(i / (len(counts) + 1)) for i in range(1, len(counts) + 1)]
        color_mapping = dict(zip(counts.index, colors))
        
        plot = sns.barplot(x=counts.values, y=counts.index, palette=color_mapping, order=counts.index, hue=counts.index, dodge=False, legend=False)
        
        for p in plot.patches:
            width = p.get_width()
            if width > 0:
                plt.annotate(f'{int(width)}', (p.get_x() + width, p.get_y() + p.get_height() / 2), ha='left', va='center', xytext=(5, 0), textcoords='offset points')
            
        # Add title and labels
        plt.title('Total Attrition by Education Level')
        plt.ylabel('')

        # Remove x-ticks for a cleaner look
        plt.xticks([])   


        ### 3rd row 2nd chart: Total Attrition by Years at Company
        plt.subplot(2, 3, 5)

        # Remove the background color for the plot
        plt.gca().patch.set_alpha(0)

        # Filter data for attrition cases
        attrition_data = filtered_df[filtered_df['Attrition'] == 'Yes']
        counts = attrition_data['Years at Company'].value_counts().sort_index()

        cmap = plt.get_cmap('Purples')
        colors = [cmap(i / (len(counts) + 1)) for i in range(1, len(counts) + 1)]

        plot = sns.barplot(x=counts.index, y=counts.values, palette=colors, hue=counts.index, dodge=False, legend=False)

        for p in plot.patches:
            height = p.get_height()
            if height > 0:  # Add annotation only if the height is greater than 0
                plot.annotate(f'{int(height)}', (p.get_x() + p.get_width() / 2., height), ha='center', va='center', xytext=(0, 10), textcoords='offset points')


        # Add title and labels
        plt.title('Total Attrition by Years at Company')
        plt.xlabel('')
        plt.ylabel('')

        # Remove y-ticks for a cleaner look
        plt.yticks([])     

        ### 3rd row 3rd chart: Distribution of Years at Company vs Attrition
        plt.subplot(2, 3, 6)

        # Remove the background color for the plot
        plt.gca().patch.set_alpha(0)

        sns.violinplot(data=filtered_df, x='Attrition', y='Years at Company', split=True, hue='Attrition', inner="quart", palette="Purples")

        # Add title and labels
        plt.title('Distribution of Years at Company vs Attrition')
        plt.ylabel('')
        plt.xlabel('')

        plt.tight_layout()
        plt.show()
    
    # Enable the widgets and hide the spinner after charts are generated
    enable_widgets()
    loading_spinner.layout.visibility = 'hidden'

# Attach the update function to filter changes
department_filter.observe(update_dashboard, names='value')
for checkbox in gender_checkboxes:
    checkbox.observe(update_dashboard, names='value')
    checkbox.observe(on_gender_checkbox_change, names='value')
age_slider.observe(update_dashboard, names='value')

# Display the output area
display(output)

# Initial dashboard load
update_dashboard()